In [0]:
from pyspark.sql.functions import count, year, desc

#### Create games dataframe from delta table

In [0]:
games_df = spark.read.table("steam_processed.games")

In [0]:
display(games_df)

appid,name,release_date,price,currency,short_description,header_image,windows,mac,linux,metacritic_score,developers,publishers,categories,genres,create_timestamp,update_timestamp
1800730,I MAED A GAM3 W1TH Z0MB1ES 1N IT!!!1,2021-11-11,0.0,EUR,A top down twin stick zombie slaying shooter that you are sure to play again and again!,https://cdn.akamai.steamstatic.com/steam/apps/1800730/header.jpg?t=1636653238,true,false,true,0,"List(Ska Studios, OMNIViOLET)",List(Ska Studios),"List(Single-player, Multi-player, Co-op, Shared/Split Screen Co-op, Shared/Split Screen, Full controller support, Remote Play Together)","List(Action, Casual, Free to Play, Indie)",2022-09-11T13:15:34.164+0000,
1800780,Find Her in Dungeon (3D Quest),2021-11-30,0.79,EUR,"Find Her in Dungeon - 3D Quest with a variety of actions, where U have to solve (as in quests) your further passage. Sometimes it's fun, sometimes it's scary, sometimes it's sad. With the help of hints and logic, try to find her.",https://cdn.akamai.steamstatic.com/steam/apps/1800780/header.jpg?t=1638339294,true,false,false,0,List(Hedgehog),List(Hedgehog Games),List(Single-player),"List(Adventure, Indie)",2022-09-11T13:15:34.164+0000,
1800830,Russian Drunken Boxers 2,2021-11-12,8.99,EUR,New fights of drunken Russian boxers at night. Spoof your friend or Ai bot using empty bottles of alcohol. Use super abilities,https://cdn.akamai.steamstatic.com/steam/apps/1800830/header.jpg?t=1654492377,true,false,false,0,List(Svarovsky),List(Gales Corp.),"List(Single-player, Multi-player, Co-op, Shared/Split Screen Co-op, Shared/Split Screen, Steam Achievements, Remote Play Together)","List(Action, Indie, Sports)",2022-09-11T13:15:34.164+0000,
1800900,SpaceGaze,2021-11-15,11.59,EUR,SpaceGaze is a space arena shooter in which you have to destroy hordes of space invaders from another universe.,https://cdn.akamai.steamstatic.com/steam/apps/1800900/header.jpg?t=1650352712,true,false,false,0,List(galesoozka),List(galesoozka),"List(Single-player, Steam Achievements)","List(Indie, Simulation, Sports)",2022-09-11T13:15:34.164+0000,
1800920,Medusa's Heart of Stone Chapter 01,2022-03-15,0.0,EUR,"Medusa's Heart of Stone is a 2D narrative driven game with action sequences, puzzles and quest interspersed throughout. Feared by mortals and forsaken by the gods, Medusa embarks on a journey to find someone who could love her.",https://cdn.akamai.steamstatic.com/steam/apps/1800920/header.jpg?t=1647374260,true,false,false,0,List(Tired Moon Studios),List(Milpitians),List(Single-player),"List(Adventure, Indie)",2022-09-11T13:15:34.164+0000,
1800970,Arcade Galaxy,2021-11-12,8.19,EUR,"Arcade Galaxy - a game made in 2D-hand-drawn graphics, which is a space arcade in a retro style.",https://cdn.akamai.steamstatic.com/steam/apps/1800970/header.jpg?t=1636744205,true,false,false,0,List(Crooked Road),List(Piece Of Voxel),"List(Single-player, Multi-player, Co-op, Shared/Split Screen Co-op, Shared/Split Screen, Remote Play Together)","List(Action, Adventure, Indie, Simulation)",2022-09-11T13:15:34.164+0000,
1800980,Fish Story,2021-11-11,0.79,EUR,Atmospheric underwater game for the whole family! Help the brave fish solve the mystery of the lights!,https://cdn.akamai.steamstatic.com/steam/apps/1800980/header.jpg?t=1643887804,true,false,true,0,List(Five Percents),List(Five Percents),"List(Single-player, Steam Achievements, Steam Cloud)","List(Action, Adventure, Casual, Indie)",2022-09-11T13:15:34.164+0000,
1801000,Traffic Racing,2021-11-12,8.99,EUR,"Traffic Racing is a 3D car simulator, you play as the racer Ron, on the highway, overtake as many cars as possible, and do not get off the highway",https://cdn.akamai.steamstatic.com/steam/apps/1801000/header.jpg?t=1654492888,true,false,false,0,List(REALBB),List(Gales Corp.),List(Single-player),"List(Casual, Indie, Simulation)",2022-09-11T13:15:34.164+0000,
1801060,Helix,2021-11-22,0.0,EUR,"Helix is a low-poly multiplayer FPS Hero-Shooter with MOBA-like Mechanics where during a match you will ha

#### Select wanted columns with only year from release date

In [0]:
selected_games_df = games_df.select(
                                    games_df["appid"],
                                    year(games_df["release_date"]).alias("release_year"))

#### Create list of games by year

In [0]:
game_list_by_year = selected_games_df.groupBy("release_year").agg(count("appid").alias("games_per_year")).orderBy(desc("release_year")).filter("release_year >= 2010")

In [0]:
display(game_list_by_year)

release_year,games_per_year
2022,1431
2021,3911


#### Write to delta table

In [0]:
game_list_by_year.write.mode("overwrite").format("delta").saveAsTable("steam_presentation.game_list_by_year")

#### Test table

In [0]:
%sql
SELECT * FROM steam_presentation.game_list_by_year WHERE release_year >= 2010 ORDER BY release_year;

release_year,games_per_year
2011,2
2012,2
2013,3
2014,5
2015,11
2016,15
2017,35
2018,36
2019,27
2020,55
